In [2]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset

In [4]:
with Dataset('../../data/Nyles/plume_ref4/plume_ref4_00_hist.nc', 'r') as nc:
    #print(nc.variables)
    b = np.array(nc['b'][:])
    x = np.array(nc['x'][:])
    y = np.array(nc['y'][:])
    z = np.array(nc['z'][:])
    #w = np.array(nc['w'][:])
    #v = np.array(nc['v'][:])
    #u = np.array(nc['u'][:])
    